In [2]:
%matplotlib inline
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from fbprophet import Prophet
import pandas as pd
import numpy as np
import pyodbc
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.legend

import matplotlib.pyplot as plt
import pandas.io.sql as psql

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
cnxn = pyodbc.connect('xxxxxxx')
cursor = cnxn.cursor()

Error: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found, and no default driver specified (0) (SQLDriverConnect)')

In [10]:
sql = '''SELECT * FROM dbo.Dim_Borough'''
cursor.execute(sql)

In [8]:
sql_req_meters_BBL = '''
SELECT Distinct(BBL)
FROM dbo.Fact_Consumption
WHERE Meter_No = '{0}'
'''.format('O75744368')
meter_BBL = pd.read_sql(sql_req_meters_BBL, cnxn)
#meter_BBL['BBL'][0]

In [9]:
sql_req_unique_reads = '''
SELECT COUNT(Distinct ConsumptionCF) FROM dbo.Fact_Consumption
WHERE (ConsumptionDate BETWEEN '2013-03-01 00:00:00' AND '2016-01-01 00:00:00'
AND BBL = '{0}')'''.format(meter_BBL['BBL'][0])
unique_cons = pd.read_sql(sql_req_unique_reads, cnxn)
#double check whether this meter is faulty and does not have substantial unique consumption values

In [21]:
sql_req_MeterRep = '''
SELECT LMR_Dt, Old_Meter_No, New_Meter_No FROM dbo.MetrsReplaced WHERE
((Old_Meter_No = '{0}' OR New_Meter_No = '{0}')
AND LMR_Dt BETWEEN '2008-01-01 00:00:00' AND '2016-01-01 00:00:00');
'''.format('O75744368')
meter_rep = pd.read_sql(sql_req_MeterRep, cnxn)
meter_rep["LMR_Dt"] = pd.to_datetime(meter_rep["LMR_Dt"])
#check for last meter replacement date

In [23]:
sql_req_1 = '''
SELECT ConsumptionCF AS y, ConsumptionDate AS ds FROM dbo.Fact_Consumption
WHERE (ConsumptionDate BETWEEN '2013-03-01 00:00:00' AND '2016-01-01 00:00:00'
AND ConsumptionCF <> 0
AND BBL = '{0}')'''.format(meter_BBL['BBL'][0])
cons1 = pd.read_sql(sql_req_1, cnxn)
#request training data

In [25]:
sql_req_2 = '''
SELECT ConsumptionCF  AS y, ConsumptionDate AS ds FROM dbo.Fact_Consumption
WHERE (ConsumptionDate > '2016-01-01 00:00:00'
AND ConsumptionCF <> 0
AND BBL = '{0}')'''.format(meter_BBL['BBL'][0])
cons2 = pd.read_sql(sql_req_2, cnxn)
#request test data

In [26]:
cons1["ds"] = pd.to_datetime(cons1["ds"])
cons1["ds"] = (cons1["ds"]).dt.date
cons2["ds"] = pd.to_datetime(cons2["ds"])
cons2["ds"] = (cons2["ds"]).dt.date
#formatting

In [27]:
cons1a = cons1
cons2a = cons2

In [28]:
cons1a_92 = cons1a.y.quantile(.92)
cons1a_8 = cons1a.y.quantile(.08)
cons1a = cons1a.loc[cons1a['y'] < cons1a_92]
cons1a = cons1a.loc[cons1a['y'] > cons1a_8]
#trimming erratic values (top and bottom .08 quartile)

In [29]:
cons2a_95 = cons1a.y.quantile(.95)
cons2a_5 = cons1a.y.quantile(.05)
cons2a = cons2a.loc[cons2a['y'] < cons1a_95]
cons2a = cons2a.loc[cons2a['y'] > cons1a_5]
#trimming erratic values (top and bottom .05 quartile)

In [30]:
fb_cons1 = pd.DataFrame(cons1a.groupby(['ds'])['y'].sum())
fb_cons1['ds'] = fb_cons1.index
fb_cons1 = fb_cons1.reset_index(drop=True)
fb_cons1=fb_cons1.sort_values('ds')
fb_cons1["ds"] = pd.to_datetime(fb_cons1["ds"])
fb_cons2=cons2a[['ds','y']]
fb_cons2 = pd.DataFrame(fb_cons2.groupby(['ds'])['y'].sum())
fb_cons2['ds'] = fb_cons2.index
fb_cons2 = fb_cons2.reset_index(drop=True)
fb_cons2=fb_cons2.sort_values('ds')
fb_cons2["ds"] = pd.to_datetime(fb_cons2["ds"])
#formatting training and test data for fbprophet

In [31]:
if (len(meter_rep) > 0):
    if( meter_rep["LMR_Dt"][0] > fb_cons1[['ds']].min()[0]):
        m = Prophet(yearly_seasonality=True,changepoints=[meter_rep["LMR_Dt"][0]])
    else:
        m = Prophet(yearly_seasonality=True)
else:
    m = Prophet(yearly_seasonality=True)
#incorporate last meter replacement date as a changepoint

In [32]:
m.fit(fb_cons1)
future = m.make_future_dataframe(periods=(fb_cons2.shape[0]+1))
#ensure the forecast is for the same number of days as the test data

In [33]:
forecast = m.predict(future)

In [35]:
predict_2016 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
cons_2016 = pd.concat([fb_cons1,fb_cons2],axis=0)
cons_2016["ds"] = pd.to_datetime(cons_2016["ds"])
all_2016 = pd.merge(cons_2016,predict_2016,left_on='ds',right_on='ds',how='left')
#concatenated table of date, predicted data, and confidence intervals for training and forecast data

In [36]:
t1=pd.DataFrame({'year': [2016],'month': [1],'day': [1]})
t1 = pd.to_datetime(t1).dt.date
#beginning of forecast data

In [37]:
y_16_true = all_2016.loc[all_2016['ds'] >= t1[0], ['y','ds']]
y_16_predict = all_2016.loc[all_2016['ds'] >= t1[0], ['yhat','ds','yhat_lower','yhat_upper']]
y_train_true = all_2016.loc[all_2016['ds'] < t1[0], ['y','ds']]
y_train_predict = all_2016.loc[all_2016['ds'] < t1[0], ['yhat','ds','yhat_lower','yhat_upper']]
#extracting variables for calculating mean absolute error

In [38]:
mae_test = mean_absolute_error(y_16_true.y, y_16_predict.yhat)
yyhatmean = ((y_16_true['y']+y_16_predict['yhat']).mean())/2
test_mae = ("{:.2%}".format(mae_test/yyhatmean))
#mean absolute error for forecast data

In [39]:
mae_train = mean_absolute_error(y_train_true.y, y_train_predict.yhat)
yyhatmean_train = ((y_train_true['y']+y_train_predict['yhat']).mean())/2
train_mae = ("{:.2%}".format(mae_train/yyhatmean_train))
#mean absolute error for training data

In [40]:
all_2016 = all_2016.sort_values('ds')

In [41]:
yhl_train = y_train_predict.yhat_lower.tolist()
yhu_train = y_train_predict.yhat_upper.tolist()
dates_train = y_train_predict.ds.tolist()
yhl_predict = y_16_predict.yhat_lower.tolist()
yhu_predict = y_16_predict.yhat_upper.tolist()
dates_predict = y_16_predict.ds.tolist()
#extracting variables for plottiing

In [1]:
fig = plt.figure(figsize=())

plt.plot(all_2016.ds,all_2016.y,'.k',label='Observed consumption')
plt.plot(y_train_predict.ds,y_train_predict.yhat,color = '#003399',label='Fit cons.')
plt.plot(y_16_predict.ds,y_16_predict.yhat,color = '#009900',label='Forecast cons.')
plt.fill_between(dates_train,yhl_train,yhu_train,color = '#66CCFF', alpha=0.7,label='Fit confidence interval')
plt.fill_between(dates_predict,yhl_predict,yhu_predict,color = '#28E528', alpha = 0.7,label='Forecast confidence interval')

plt.ylabel('Consumption (Cubic Feet)')
plt.ylim((min(y_16_predict.yhat_lower.min()-(3*np.std(y_16_predict.yhat_lower)),
     y_train_predict.yhat_lower.min()-(3*np.std(y_train_predict.yhat_lower))),
         max (y_16_predict.yhat_upper.max()+(3*np.std(y_16_predict.yhat_upper)),
     y_train_predict.yhat_upper.max()+(3*np.std(y_train_predict.yhat_upper)))))

plt.legend(loc='best')

plt.show()

NameError: name 'plt' is not defined

In [21]:
sql_req_bldg_desc = '''
SELECT CATEGORY_ID, DESCRIPTION
FROM dbo.Building_Class_Categories'''
bldg_desc = pd.read_sql(sql_req_bldg_desc, cnxn)
bldg_desc = bldg_desc.to_dict(orient='dict')

In [22]:
bldg_desc

{u'CATEGORY_ID': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31},
 u'DESCRIPTION': {0: u'1-Family                                          ',
  1: u'2-Family                                          ',
  2: u'3-Family                                          ',
  3: u'Multi-Family                                      ',
  4: u'Elevator Apts                                     ',
  5: u'Warehouses                                        ',
  6: u'Factory & Industrial                              ',
  7: u'Garages & Gas Stations                            ',
  8: u'Hotels                                            ',
  9: u'Hospitals & Health                                ',
  10: u'Theaters                                          ',
  11: u'Store B